In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'


In [ ]:
import sys, os, distutils
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))


In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.1 ; cuda:  cu121
detectron2: 0.6


In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
#uploading csv file
import io
import pandas as pd
from google.colab import files
uploaded = files.upload()


In [ ]:
import cv2
import tqdm
import pandas as pd
from detectron2.utils.visualizer import ColorMode, Visualizer

# Setup for Detectron2 predictor
# ...

# Initialize your video reader
im = cv2.VideoCapture('/content/vid7.mp4')
num_frames = int(im.get(cv2.CAP_PROP_FRAME_COUNT))

# Initialize a list to collect data
data = []

def runOnVideo(im, maxFrames):
    readFrames = 0
    while True:
        hasFrame, frame = im.read()
        if not hasFrame or readFrames >= maxFrames:
            print("noframe")
            break

        outputs = predictor(frame)

        if outputs['instances'].has("pred_classes") and outputs['instances'].has("scores"):
            classes = outputs['instances'].pred_classes.to('cpu').numpy()
            confidence_score = outputs['instances'].scores.to('cpu').numpy()

            metadata = predictor.metadata
            class_names = [metadata.thing_classes[class_id] for class_id in classes]

            for class_name, score in zip(class_names, confidence_score):
                data.append({'Frame': readFrames + 1, 'Object': class_name, 'Confidence': score})

        readFrames += 1

# Process video and collect data
runOnVideo(im, num_frames)

# Convert collected data to DataFrame and save as CSV
df = pd.DataFrame(data, columns=['Frame', 'Object', 'Confidence'])
df.to_csv('/content/dataframe.csv', index=False)

# Release video reader resources
im.release()


This is to seperate every frames in the video

# New Section